# smolagents: Building Intelligent AI Agents with Open Source LLMs

<img src="./media/smolagents.jpg" width=600>

From the developers at Hugging Face comes [smolagents](https://huggingface.co/blog/smolagents), a powerful yet streamlined framework for building AI agent systems using open source large language models. The framework emphasizes simplicity while providing a robust, modular, and scalable approach to agentic llm system design.

**Key Features:**
- Native integration with [transformers](https://huggingface.co/docs/transformers/en/index) ecosystem 
- Local deployment support via [Ollama](https://ollama.com/)
- Cloud flexibility through [HuggingFace API](https://huggingface.co/docs/api-inference/index)
- Broad model compatibility via [LiteLLM](https://docs.litellm.ai/) and OpenAI API specs

Resources: [Documentation](https://huggingface.co/docs/smolagents/index) | [GitHub Repository](https://github.com/huggingface/smolagents/tree/main)

**This notebook provides a comprehensive walkthrough of smolagents, covering:**
1. Understanding Agents and Code Agents
2. Implementing Agents with Different Backends
   - HuggingFace Inference API
   - OpenAI API Compatible Services  
   - Local Deployment with Ollama
   - Transformers Integration
3. Building Custom Tools and Actions
4. Advanced Features
   - Intermediate Planning Steps
   - Multi-Agent Architectures
5. Creating User Interfaces with Gradio

---
## Dependencies

In [ ]:
from smolagents import (
    CodeAgent, 
    ManagedAgent,
    HfApiModel,
    tool,
    Tool,
    DuckDuckGoSearchTool,
    load_tool,
    GradioUI,
    LiteLLMModel
)

In [5]:
from huggingface_hub import notebook_login

notebook_login()

---
## Single Agents

Smolagents defines AI agents as **programs where LLM outputs directly control the workflow**. Different levels of agent autonomy and capability are possible:

<img src="./media/agents_table.png" width=600>

At their core, agents rely on [tools](https://platform.openai.com/docs/guides/function-calling) - defined functions that allow LLMs to interact with their environment through function execution and output observation. This enables agents to take actions and learn from their results.

To orchestrate this interaction, smolagents employs the [ReAct](https://react-lm.github.io/) (Reason + Act) framework, which follows the process:

1. **Input Phase**: User query, context, and response instructions are processed
2. **Reasoning Phase**: LLM generates a "thought", planning next steps based on context
3. **Action Phase**: LLM executes a [tool call](https://youtu.be/gMeTK6zzaO4?si=XzwYDe0cVA_hRoaZ), triggering a function, API or sequence
4. **Observation Phase**: Tool output is returned to the LLM for processing
5. **Loop**: Steps 2-4 repeat until either:
   - Original question is answered
   - Maximum steps are reached 
   - Final answer tool is called

<img src="./media/ReAct.png" width=600> 

While this approach is commonly used with Tool Calling Agents like ChatGPT through JSON-based responses, smolagents supports both traditional tool calling via `ToolCallingAgent` and a more powerful code-first default approach.

## CodeAgents

Traditional agent implementations use JSON-formatted function calls, where LLMs generate structured objects with function names and arguments. While functional, this approach has key limitations identified in [DynaSaur: Large Language Agents Beyond Predefined Actions](https://arxiv.org/pdf/2411.01747):

1. Restricted action space through fixed function sets
2. High maintenance burden for implementing and maintaining individual actions
3. Limited ability to compose multiple actions together
4. No native support for intermediate state management
5. Increased complexity in multi-step problem solving

To address these limitations, smolagents enables direct code execution for tool handling:

<img src="./media/codeagent.png" width=800>

As demonstrated in [Executable Code Actions Elicit Better LLM Agents](https://arxiv.org/pdf/2402.01030), code-based execution provides natural support for:
- Control flow (if-statements, loops)
- Data flow (variable storage, passing)
- Tool composition (chaining actions)
- State management (intermediate results)
- Error handling (try/catch blocks)

The example above shows how CodeAct (top right) can handle multiple inputs in a single action through loops, while JSON/text approaches (top left) require separate actions for each input.

By defining tools as Python functions and allowing LLMs to write complete programs, smolagents offers several advantages:

1. **Ecosystem Integration**: Direct access to powerful [python packages](https://github.com/huggingface/smolagents/blob/main/src/smolagents/utils.py#L36)
2. **Natural LLM Interaction**: Leverages models' advanced code generation and understanding capabilities
3. **Efficient Execution**: Complex orchestration with fewer discrete actions
4. **Robust State Management**: Variable handling for intermediate results
5. **Real-time Feedback**: Immediate error handling during execution

<img src="./media/codeactinstruct.png" width=800>

### Hugging Face API

<img src="./media/hf.png" width=400>

The [Hugging Face Inference API](https://huggingface.co/docs/api-inference/index) provides cloud access to most currently popular models hosted on Hugging Face. With flexible rate limits and pay-per-call pricing based on subscription tier, it's an excellent starting point for working with open source models. See the [documentation](https://huggingface.co/docs/api-inference/supported-models) for detailed model support information.

Given our focus on code execution, we'll use [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct) as our base model. By adding `add_base_tools` we get instant access to web search and page loading capabilities.

In [ ]:
# Define the Model
hfapi_model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

# Create the Agent
hfapi_agent = CodeAgent(tools=[], 
                        model=hfapi_model, 
                        add_base_tools=True, 
                        verbosity_level=2 # See full logs
                       )

# Launch the Agent
hfapi_agent.run(
    "Who is Adam Lucek the AI youtuber?",
)

#### Behind the Scenes

Smolagents handles prompting between steps very eloquently, it's worth reading over their [prompts.py](https://github.com/huggingface/smolagents/blob/main/src/smolagents/prompts.py) file to see exactly how each LLM call is handled across the sequence.

Let's take a peek at some of the intermediate steps not shown

**System Prompt**

In [7]:
print(hfapi_agent.logs[0].system_prompt)

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

**User Message**

This is placed after the system prompt

In [8]:
print(hfapi_agent.logs[2].agent_memory[1]['content'])

New task:
Who is Adam Lucek the AI youtuber?


**Agent Response**

In [9]:
print(hfapi_agent.logs[3].agent_memory[2]['content'])

Thought: To find information about Adam Lucek, the AI YouTuber, I will perform a web search to gather relevant details.

Code:
```py
adam_lucek_info = web_search(query="Adam Lucek AI YouTuber")
print(adam_lucek_info)
```<end_code>


**Final Response**

Skipping a few in between calls and observations for brevity

In [10]:
print(hfapi_agent.logs[8].error, "\n\n")
print(hfapi_agent.logs[8].action_output)

Reached max steps. 


Based on the available information from the search results, Adam Lucek appears to be an AI YouTuber who focuses on AI and machine learning topics. Here are some key points about him:

1. **YouTube Channel**: Adam Lucek has a YouTube channel where he shares content related to AI, including videos on fine-tuning language models, multimodal retrieval augmented generation (RAG), and other AI-related projects.

2. **LinkedIn Profile**: Adam Łucek, likely the same person, has a LinkedIn profile where he is listed as a MarTech Portfolio & Innovation Specialist at Cisco. His LinkedIn posts mention his work with AI and generative models, such as creating AI-powered robots and discussing the impact of generative AI on his YouTube videos.

3. **GitHub and Hugging Face**: Adam Łucek is active on GitHub and Hugging Face, sharing his projects and models related to machine learning and AI.

4. **Engagement**: His YouTube videos have gained significant popularity, with one of his

### OpenAI API Specification

<img src="./media/oai.png" width=300>

Many model providers have adopted the [OpenAI API Specification](https://platform.openai.com/docs/api-reference/introduction), making it easy to swap models without rewriting application code. Smolagents uses [litellm](https://docs.litellm.ai/) to handle this integration automatically across their supported [provider list](https://docs.litellm.ai/docs/providers). This means you only need to adjust the `api_base` endpoint and model name to work with any compatible API.

In [11]:
# Define the Model

# litellm_model = LiteLLMModel(model_id="openai/gpt-4o-mini")
# litellm_model = LiteLLMModel(model_id="gemini/gemini-1.5-flash")
# litellm_model = LiteLLMModel(model_id="fireworks_ai/accounts/fireworks/models/llama-v3p3-70b-instruct")

litellm_model = LiteLLMModel(model_id="anthropic/claude-3-5-haiku-latest")

# Create the Agent
litellm_agent = CodeAgent(tools=[], 
                  model=litellm_model, 
                  add_base_tools=True)

# Launch the Agent
litellm_agent.run(
    "What is Adam Lucek's subscriber count multiplied by the 118th fibbonaci sequence number",
)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ What is Adam Lucek's subscriber count multiplied by the 118th fibbonaci sequence number                                                                                                              │
│                                                                                                                                                                                                      │
╰─ LiteLLMModel - anthropic/claude-3-5-haiku-latest ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Adam Lucek's subscriber count                                                                                                                                                            
  search_result = web_search(query="Adam Lucek YouTube subscribers")                                                                                                                                    
  print("Search result:", search_result)                                                                                                                                                                
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Search result: ## Search Results

[Adam Lucek - YouTube](https://www.youtube.com/channel/UCy6rH4hKHMLdd5s-FUrto7Q/community?lb=UgkxspARMV7P9GwIDZ4E4f_cRMtbrcxkxnxm)
teach them to long for the endless immensity of the sea Documenting my learning journey with the world For inquiries, refer to the email in my links section.

[Post from Adam Lucek - YouTube](https://www.youtube.com/post/UgkxZ75ueO4LcqO-Qq6LqyVgvkrrCf5oUlF6)
Thank you all so much for 4,000 subscribers 🎉🎉🎉🥳🥳🥳🥳 I appreciate all of the support and positivity that's come from this channel so far, here's to more! As a...

[Adam Lucek - YouTube](https://www.youtube.com/channel/UCLFzcap1jIaiM6KRG90Dwbw/about)
Share your videos with friends, family, and the world

[Track YouTube analytics, future predictions, & live subscriber counts ...](https://socialblade.com/youtube/)
View the daily YouTube analytics, track progress charts, view future predictions, related channels, and track realtime live sub counts. ... Top 25 YouTube Users by Subscribers View More. Rank. Grade. 
Display Name. Videos. Subscribers.

[AdamLucek/youtube-titles · Datasets at Hugging Face](https://huggingface.co/datasets/AdamLucek/youtube-titles)
You look at a given youtube title and description and reverse it to a plausible prompt that would cause a title generating assistant to create that title. Note that youtube descriptions are not all 
directly related to the video, and often contain advertisements irrelevant information to the actual video.

[Lucek's YouTube Stats and Insights - vidIQ](https://vidiq.com/youtube-stats/channel/UCbdNs5JkKaqvn623qrydqBw/)
View the daily YouTube analytics of Lucek to get AI-powered insights based on progress charts, view future predictions, related channels, engagement rates, and more. Features. Features. Daily Ideas; 
Keyword Tools; ... Top 50 YouTube Channels. Top 100 YouTube Channels. Top 500 YouTube Channels.

[Lucek Youtuber overview](https://us.youtubers.me/mowpolucku/youtuber-stats)
Purple Daily - a Minnesota Vikings Podcast. 40,500 Subscribers. gaming chair

[Adam Lucek - Invidious](https://yt.houtworm.vip/channel/UCy6rH4hKHMLdd5s-FUrto7Q)
View channel on YouTube. Switch Invidious Instance. Videos. Playlists. Community. newest. oldest. popular. 24:46. ... Adam Lucek Shared 1 month ago. 10K views. 9:30. Speak Any Language With AI - 
Realtime Speech-to-Speech Translation & Voice Synthesis (w/Code) Adam Lucek Shared 1 month ago ...

[Adam Łucek - MarTech Portfolio & Innovation Specialist - Cisco - LinkedIn](https://www.linkedin.com/in/adamrlucek)
Adam's projects were in the digital marketing realm of selector tools, filtering and social media platforms, masterfully presented to hundreds, 100% data driven, employing swiftly adopted ...

[Post from Adam Lucek - YouTube](https://www.youtube.com/post/Ugkxv0181gV3cvZRNAaAiRh8MpCtOeIdWEd3)
Ever thought about merging already fine tuned language models?🤔 🤖🤝🤖🟰🦾 What it is, ways to do it, and examples of 4 different methods across 4 different archi...

Out: None

[Step 0: Duration 3.72 seconds| Input tokens: 2,405 | Output tokens: 94]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  # Multiply subscriber count by 118th Fibonacci number                                                                                                                                                 
  result = 4000 * 2046711111473984623691759                                                                                                                                                             
                                                                                                                                                                                                        
  # Use final_answer to submit the result                                                                                                                                                               
  final_answer(result)                                                                                                                                                                                  
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 8186844445895938494767036000

[Step 2: Duration 4.69 seconds| Input tokens: 9,984 | Output tokens: 366]

8186844445895938494767036000

### Ollama Local Deployment

<img src="./media/ollama.png" width=150>

For local deployment of quantized LLMs, [Ollama](https://ollama.com/) has become the go-to solution. Since Ollama provides an OpenAI API compatible endpoint at localhost:11434, we can use LiteLLM once again to seamlessly integrate with locally running models, bringing cloud-like capabilities to your local machine.

In [ ]:
# Define the Model
ollama_model = LiteLLMModel(
    model_id="ollama_chat/llama3.2:1b",
    api_base="http://localhost:11434"
)

# Create the agent
ollama_agent = CodeAgent(tools=[], 
                  model=ollama_model, 
                  add_base_tools=True)

# Launch the Agent
ollama_agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

### Transformers Direct Access

<img src="./media/hft.png" width=300>

Smolagents integrates directly with [Hugging Face Transformers](https://huggingface.co/docs/transformers/index), enabling any compatible model to be downloaded and used locally. This opens up vast possibilities for open source agent development, as smolagents can work with any transformers model. At the time of writing, there are over [480,000](https://huggingface.co/models?library=transformers) compatible models available!

In [ ]:
# Define the Model
transformers_model = TransformersModel(model_id="meta-llama/Llama-3.2-3B-Instruct")

# Create the Agent
transformers_agent = CodeAgent(tools=[], 
                  model=transformers_model, 
                  add_base_tools=True,
                  additional_authorized_imports=["numpy"]
)

# Launch the Agent
transformers_agent.run(
    "Find the eigenvalues of: [[4, -2],[1, 1]]",
)

*Note: Many models accessed via Transformers require GPU acceleration, check out this above example in [Google Colab](https://colab.research.google.com/drive/1FrmfDRs1S8jwj3bt9K-CUniMG8zgxsEf?usp=sharing).*

---
## Tools

<img src="./media/llm_tools.png" width=600>

*[Tool Calling for LLMs: A Detailed Tutorial](https://medium.com/@developer.yasir.pk/tool-calling-for-llms-a-detailed-tutorial-a2b4d78633e2)*

Tools are the actions that the LLM can choose to take, configure, or use to solve a problem. Being able to interact with an environment through different actions defined as tools is what takes LLMs from simple text generators to useful agents. While tools have traditionally been defined as [json schemas](https://platform.openai.com/docs/guides/function-calling) that get parsed and executed, smolagents uses the aforementioned code execution method. This makes creating tools much easier, as they are going to be literal functions.

Smolagents comes with a few [built in tools](https://github.com/huggingface/smolagents/blob/main/src/smolagents/default_tools.py):
1. A `DuckDuckGoSearchTool` for searching the web with Duck Duck Go
2. A `GoogleSearchTool` for searching the web with Google
3. A `VisitWebPageTool` for loading webpage URL's in markdown format
4. A `SpeechToTextTool` for speech to text using OpenAI's Whisper Model using Transformers pipelines

In [13]:
search_tool = DuckDuckGoSearchTool()
print(search_tool("How many McDonalds are in New York City?"))

## Search Results

[How Many Mcdonald'S Are There In New York City?](https://www.eyeandpen.com/how-many-mcdonalds-in-new-york-city/)
But just how many McDonald's are in New York City? Read on to find out the surprising answer. A Quick Answer If you're short on time, here's a quick answer: there are approximately 250 McDonald's restaurants currently operating within New York City's five boroughs - Manhattan, Brooklyn, Queens, Staten Island, and the Bronx. ...

[How Many McDonald's Are In NYC (Quick Information)](https://sarasveggiekitchen.com/how-many-mcdonalds-are-in-nyc/)
When Was The First McDonald's Opened In New York City. The first McDonald's restaurant in New York City was opened by Lee Dunham in Harlem, it was located at 215 West 125th Street. At the time burgers cost 23 cents and fries were only 15 cents. Why Does McDonald's Put Pickles On It's Burgers (All About McDonald's Pickles)

[How many mcdonalds are in nyc? - GB Times](https://gbtimes.com/how-many-mcdonalds-are-in-nyc/)

But creating your own is very straightforward!

### Tool Decorator

The simplest way to convert a function into a usable tool is with the `@tool` decorator, which will handle converting a function into an executable function that the LLM can understand and use.

As the decorator is converting the function into a description that the LLM can understand, you need a few key components:
1. A clear name
2. Type hints on both inputs and output
3. A description, that includes an ‘Args:’ part where each argument is described

In [14]:
import requests

@tool
def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    """
    Convert an amount from one currency to another using fawazahmed0's free API.
    
    Args:
       amount: The amount to convert
       from_currency: The source currency code (e.g., 'USD', 'EUR', 'BTC')
       to_currency: The target currency code (e.g., 'USD', 'EUR', 'BTC')
    """
    url = f'https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{from_currency.lower()}.json'
    
    response = requests.get(url)
    rates = response.json()[from_currency.lower()]
    converted = amount * rates[to_currency.lower()]
    
    return round(converted, 2)
    

# Example usage:
result = convert_currency(100, 'USD', 'EUR')
print(result)

97.05


In [15]:
print("\n=== Currency Converter Function Details ===\n")
print(f"Function Name: {convert_currency.name}")
print(f"Description: {convert_currency.description}")
print("\nInput Parameters:")
for param_name, param_info in convert_currency.inputs.items():
    print(f"- {param_name}: {param_info['type']} - {param_info['description']}")
print(f"\nOutput Type: {convert_currency.output_type}")


=== Currency Converter Function Details ===

Function Name: convert_currency
Description: Convert an amount from one currency to another using fawazahmed0's free API.

Input Parameters:
- amount: number - The amount to convert
- from_currency: string - The source currency code (e.g., 'USD', 'EUR', 'BTC')
- to_currency: string - The target currency code (e.g., 'USD', 'EUR', 'BTC')

Output Type: number


### Subclass Tool

On the other hand, you can subclass `Tool` [(see main class in tools.py)](https://github.com/huggingface/smolagents/blob/main/src/smolagents/tools.py) for more detailed control over your tool's inputs, outputs and behavior.

In [17]:
class ConvertCurrency(Tool):
    # Function Name
    name = "convert_currency_2"
    # Function Description
    description = "Convert an amount from one currency to another using fawazahmed0's free API."
    # Inputs with Descriptions
    inputs = {
             'amount': {'type': 'number', 
                         'description': 'The amount to convert'},
             'from_currency': {'type': 'string', 
                               'description': "The source currency code (e.g., 'USD', 'EUR', 'BTC')"},
             'to_currency': {'type': 'string', 
                             'description': "The target currency code (e.g., 'USD', 'EUR', 'BTC')"}
             }
    # Output
    output_type = "number"
    
    # Forward function
    def forward(self, amount: float, from_currency: str, to_currency: str) -> float:
        url = f'https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{from_currency.lower()}.json'
        
        response = requests.get(url)
        rates = response.json()[from_currency.lower()]
        converted = amount * rates[to_currency.lower()]
        
        return round(converted, 2)

In [18]:
currency_coverter = ConvertCurrency()
currency_coverter.forward(100, 'USD', 'EUR')

97.05

**Demonstration**

In [19]:
# Define the Model
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

# Load the Tool
currency_coverter = ConvertCurrency()

# Create the Agent
agent = CodeAgent(tools=[currency_coverter], 
                  model=model, 
                  add_base_tools=True)

# Launch the Agent
agent.run(
    "What is 500 great british pounds to dollars",
)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ What is 500 great british pounds to dollars                                                                                                                                                          │
│                                                                                                                                                                                                      │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  conversion_result = convert_currency_2(amount=500, from_currency='GBP', to_currency='USD')                                                                                                            
  print(conversion_result)                                                                                                                                                                              
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
611.66

Out: None

[Step 0: Duration 2.04 seconds| Input tokens: 2,231 | Output tokens: 69]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  conversion_result = convert_currency_2(amount=500, from_currency='GBP', to_currency='USD')                                                                                                            
  final_answer(conversion_result)                                                                                                                                                                       
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 611.66

[Step 1: Duration 1.92 seconds| Input tokens: 4,631 | Output tokens: 143]

611.66

### Tools from Hugging Face

Uniquely, smolagents implements a way to use [Hugging Face spaces](https://huggingface.co/spaces) via the gradio api as tools. 

This allows you to connect more complex hosted tools together via the entire HF ecosystem. From their documentation, they show how we can use the [Black Forest Labs FLUX.1-schnell](https://huggingface.co/spaces/black-forest-labs/FLUX.1-schnell) space as a tool.

In [21]:
from PIL import Image

# Main HF Connection
image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

# resp = image_generation_tool("A sunny beach")
# img = Image.open(resp)
# img.show()

# Additional Tool to Display Image
@tool
def display_image(path: str) -> str:
    """
    Use

    Args:
        path: output path from the image generation tool
    """
    image = Image.open(path)
    image.show()

    return "Success"

Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.


**Demonstration**

In [22]:
# Define the Model
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

# Create the Agent
agent = CodeAgent(tools=[image_generation_tool, 
                         display_image], 
                  model=model)

# Launch the Agent
agent.run(
    "generate an image of a green basketball and show it to the user"
)

╭────────────────────────────────────────────────────────────────────────────────────────────── New run ───────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                                                                                      │
│ generate an image of a green basketball and show it to the user                                                                                                                                      │
│                                                                                                                                                                                                      │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  image_path = image_generator(prompt="A green basketball")                                                                                                                                             
  print(image_path)                                                                                                                                                                                     
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
/private/var/folders/yw/8psyh6kn5nb8tbbkw580yn340000gn/T/gradio/8e5c087d73091d2e9ed83c3dcb8f8ae2fe7f90634b8d506e21e5ca052aaabd07/image.webp

Out: None

[Step 0: Duration 7.23 seconds| Input tokens: 2,066 | Output tokens: 60]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The green basketball image has been displayed.")                                                                                                                                        
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The green basketball image has been displayed.

[Step 2: Duration 1.30 seconds| Input tokens: 6,786 | Output tokens: 160]

'The green basketball image has been displayed.'

Sharing and importing tools from the Hugging Face Hub is made simple too, with the `tool_name.push_to_hub("{your_username}/{your-tool-name}")` function!

---
## Planning

<img src="./media/planning.png" width=400>

*[LLM Agents](https://www.promptingguide.ai/research/llm-agents)*

Optionally, you can instruct an agent to include one or multiple planning steps during its execution. This can help with complex multi-step tasks, or when the agent needs to adapt its approach based on intermediate results.

When an agent performs planning, it breaks down a task into two key components:

1. Facts List: Understanding what is known and what needs to be discovered
2. Action Plan: Creating a step-by-step approach to solve the task

The planning process happens through two main steps:

1. **Initial Planning** (when step=0):
- The agent first analyzes the task to create a FACTS list categorizing:
  - Facts given in the task
  - Facts to look up
  - Facts to derive
- Then creates an initial action plan based on these facts and available tools

2. **Plan Updates** (at intervals):
- The agent updates its facts with what it has learned
- Creates a new/updated plan based on:
  - Progress so far
  - Remaining steps
  - New facts discovered
  - Available tools

Key points about planning:

- If `planning_interval=None` (default), no planning is done
- If `planning_interval=1`, planning occurs at every step
- If `planning_interval=N`, planning occurs every N steps
- The agent maintains a running list of facts and updates its plan based on what it learns
- Planning helps the agent stay organized and adapt its approach based on new information

In [ ]:
# Define the Model
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

currency_coverter = ConvertCurrency()

# Create the Agent
agent = CodeAgent(tools=[image_generation_tool, 
                         display_image,
                         currency_coverter], 
                  model = model,
                  planning_interval=3, # Planning interval
                  max_steps = 12
                 )

# Launch the Agent
agent.run(
    "Generate and display two images one of the current rate of dollars to euros on a whiteboard"
    "and another of the current rate of great british pounds to USD on a chalkboard"
)

---
## MultiAgent

<img src="./media/multi_agent.png" width=600>

*[Multi-agent LLMs in 2024](https://www.superannotate.com/blog/multi-agent-llms)*

What's better than one single agent? Multiple working together! 

As demonstrated across many frameworks now, single agents can get overwhelmed as their environment grows in complexity. Adding too many tools, situations, and pathways for an Agent to follow tends to degrade performance. Hence, the practice of creating a network of individual specialized agents that have their own subset of tools, memory, and available actions has become standard.

Smolagents supports hierarchichal multi-agent systems via their `ManagedAgent` [class](https://github.com/huggingface/smolagents/blob/main/src/smolagents/agents.py#L1072). Essentially, converting a single agent into a function itself! 

In [ ]:
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

# Currency Converter Agent
currency_converter = ConvertCurrency()
currency_agent = CodeAgent(tools=[currency_converter], model=model)
managed_currency_agent = ManagedAgent(
    agent=currency_agent,
    name="currency_agent",
    description="Converts currency from one to another using the latest exchange rate. Needs an amount, and two currencies."
)

# Image Generator Agent
image_agent = CodeAgent(tools=[image_generation_tool, display_image], model=model)
managed_image_agent = ManagedAgent(
    agent=image_agent,
    name="image_agent",
    description="Generates and displays images on the fly. Instruct it with just the prompt you want to generate"
)

# Manager Agent
manager_agent = CodeAgent(
    tools=[], model=model, managed_agents=[managed_currency_agent, managed_image_agent]
)

manager_agent.run("Generate and display two images, one of the current rate of dollars to euros on a whiteboard,"
                  "and another of the current rate of great british pounds to polish zloty on a chalkboard")

---
## Gradio Interface

<img src="./media/gradio.svg" width=400>

As a final perk of smolagents being embedded in the Hugging Face ecosystem, there is clean connection to [Gradio](https://www.gradio.app/), allowing you to provide a quick and simple frontend for your agents.

In [ ]:
# Define the Model
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

# Load the Tool
currency_coverter = ConvertCurrency()

# Create the Agent
agent = CodeAgent(tools=[currency_coverter], 
                  model=model, 
                  add_base_tools=True,
                 verbosity_level=0)

# Launch the Gradio Interface
GradioUI(agent).launch()

---
## Discussion

Smolagents provides a very robust and reusable framework for building single and multi agent language model systems. It's integration with open source providers and the Hugging Face ecosystem shows unprecedented support for building with open models, something that has been traditionally finnicky compared to the major closed providers. 

Code Agents provide a strong approach to tool use and execution, leveraging LLMs existing advanced code generation and understanding to more robustly handle and orchestrate different actions with the environment. Overall, many of the techniques used in smolagents are some of the best we've seen yet in terms of model applicability and innovative improvements to agentic performance.